In [1]:
import random
import numpy as np
import pandas as pd
import os

In [2]:
dataset_name = input("Enter the dataset name:")

In [3]:
split_criteria = input("Enter the split criteria:")

In [4]:
split_criteria = (split_criteria).split(',') 
if len(split_criteria)==1:
    split_criteria=split_criteria[0]

In [5]:
dataset_title = dataset_name
if isinstance(split_criteria, str):
    dataset_title+="_"
    dataset_title+=split_criteria
if isinstance(split_criteria, list):
    dataset_title+="_"
    for s in split_criteria:
        dataset_title+="_"
        dataset_title+=s

In [6]:
def movie_genre(genres):
    for genre in genres.split('|'):
        if genre == 'Action':
            return 0
        elif genre == 'Comedy':
            return 1
        elif genre == 'Drama':
            return 2
        else:
            return 3

In [7]:
# Pre-processing IMDB dataset
mat = pd.read_csv('IMDB_movie/movie_metadata.csv')
mat['movie_genre'] = mat['genres'].apply(movie_genre)

In [8]:
if isinstance(split_criteria, str):
    split_criteria_ = [split_criteria]
else:
    split_criteria_ = split_criteria
split_criteria_ = [x.replace('movie', 'movie_genre') for x in split_criteria_]
split_criteria_ = [x.replace('year', 'title_year') for x in split_criteria_]

In [9]:
gb = mat.groupby(split_criteria_)
key, grp = max(gb, key=lambda x: x[1].shape)
mat = grp

In [10]:
d_names, m_names, a_names = list(mat.director_name), list(mat.movie_title), list(mat.actor_1_name)
names = set(d_names+m_names+a_names)
names.discard(np.nan)
names = list(names)
indices = list(range(len(names)))
name_dir = dict(zip(names, indices))

In [11]:
os.makedirs(f'{dataset_title}/', exist_ok=True)

for index, row in mat.iterrows():
    try:
        d_name = str(name_dir[row.director_name])
        a_name = str(name_dir[row.actor_1_name])
        m_name = str(name_dir[row.movie_title])
    
        with open(f'{dataset_title}/link.dat', 'a') as w:
            w.writelines(m_name+' '+d_name+'\n')
            w.writelines(d_name+' '+m_name+'\n')
            w.writelines(m_name+' '+a_name+'\n')
            w.writelines(a_name+' '+m_name+'\n')
            
    except:
        continue

with open(f'{dataset_title}/node.dat', 'a') as w:
    for i, x in enumerate(names):
        if x in d_names:
            w.writelines(str(i)+' D'+' 0\n')
        elif x in m_names:
            w.writelines(str(i)+' M'+' 1\n')
        elif x in a_names:
            w.writelines(str(i)+' A'+' 2\n')
    